### This Notebook will create the Decision Tree algorithm from scratch. The dataset used for this will be the sklearn diabetes which is a regression problem

In [23]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
ds = datasets.load_diabetes()

In [25]:
X = np.array(ds.data)
y = np.array(ds.target)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

Now that we have the data, we need to make the decision tree. So let's look at the first step. 

The first step is to decide how we are going to separate the data. How do we find this? Since we don't know the best spli criteria, we will have to find it. We do this by iterating through each column and for each column we test every possible value to split and see which one is the best. However, if a column A can have all possible positive values, there would be infinite values for which to try to split the data. So, to circunvent this, for each column, we try to separate the labels  using only the values of that column that we found in our training data. But now, how do we determine what is the best split. There are different metrics that we can use, however, in this case, we will calculate the variance of the labels on each child node and sum them. The best split is the one that have the lower variance. 

So, let's build 5 functions:
 1) Calculates the variance of a list of values, returning a float 
 2) For a given n of labels y it sums the values of the variances, returning a float
 3) For a given column c and a list it loops through all its values and calculates the variance of the split caused by the value, returning a tuple with (value, sum of variance)
 4) For a given value v, return the indexes that are less than that value and the indexes that are equal or greater than that value, as a list of lists.
 5) For a matrix X and labels y, it loops through all the columns and executes function 3), calculating the best column and value to split on, returning a dictionary with the index of the column, the value and the variance

In [26]:
def calculate_variance(lst):
    if len(lst) == 0:
        variance = float('inf')
    else:
        total = 0
        mean = np.mean(lst)
        total = sum([(value-mean)**2 for value in lst])
        variance = total/len(lst)
    return variance

In [27]:
def calculate_lists_variances(list_of_lists:list[list[float]])->float:
    variance_sum = 0
    len_all = sum([len(ls) for ls in list_of_lists])
    variance_sum = sum([len(number_list)/len_all*calculate_variance(number_list) for number_list in list_of_lists])
    return variance_sum

In [28]:
def calculate_split_indexes(split_value:float, list_column_values:list[float])->list[list[int], list[int]]:
    indexes_smaller = []
    indexes_eq_bigger = []
    for index, value in enumerate(list_column_values):
        if value<split_value:
            indexes_smaller.append(index)
        else:
            indexes_eq_bigger.append(index)
    return [indexes_smaller, indexes_eq_bigger]


In [29]:
def calculate_best_column_variance(float_list:np.ndarray[float], labels:np.ndarray[float], min_variance:float)->tuple[float, float]:
    best_split = None
    for value in float_list:
        list_of_lists_of_indexes= calculate_split_indexes(value, float_list)
        labels_lists = [labels[indexes] for indexes in list_of_lists_of_indexes]
        value_variance = calculate_lists_variances(labels_lists)
        if value_variance< min_variance:
            min_variance = value_variance
            best_split = value
    return (best_split, min_variance)


In [30]:
def calculate_matrix_best_split(matrix: np.ndarray[float], labels: np.ndarray[float])->dict[str, float]:
    result_dictionary = {
        "column_index":-1,
        "split":-1,
        "variance":calculate_variance(labels)
    }
    for column_index in range(matrix.shape[1]):
        column_values = matrix[:, column_index] #select all values from the column of index column_index
        split_value, variance = calculate_best_column_variance(column_values, labels, result_dictionary['variance'])
        if variance<result_dictionary['variance']:
            result_dictionary['column_index'] = column_index
            result_dictionary['split'] = split_value
            result_dictionary['variance'] = variance
    return result_dictionary

### Now let's try to determine the best split for our data!

In [31]:
calculate_matrix_best_split(X_train, y_train)

{'column_index': 2,
 'split': 0.005649978676881689,
 'variance': 4288.930696708918}

Now we just need to keep doing this. Now the question is: Untill when do e keep doing this? We can do this until we can't do anymore split,(when every leaf node has only one label, multiple labels all with the same value or when the values are different but the feature rows values are all the same), when we reach a maximum number of iterations that we define, or we reach the maximum number of leaves that we define.

Let's first build one tree that does the split untill we can't do it anymore and then add the constraints. First we have to alter the function calculate_matrix_best_split and calculate_best_column_variance to also return the splits

In [32]:
def calculate_best_column_variance(float_list:np.ndarray[float], labels:np.ndarray[float], min_variance:float)->tuple[float, float]:
    best_split = None
    indexes_1 = np.empty((0))
    indexes_2 = np.empty((0))
    for value in float_list:
        list_of_lists_of_indexes= calculate_split_indexes(value, float_list)
        labels_lists = [labels[indexes] for indexes in list_of_lists_of_indexes]
        value_variance = calculate_lists_variances(labels_lists)
        if value_variance< min_variance:
            min_variance = value_variance
            best_split = value
            indexes_1 = list_of_lists_of_indexes[0]
            indexes_2 = list_of_lists_of_indexes[1]
    return (best_split, min_variance,indexes_1, indexes_2)


In [33]:
def calculate_matrix_best_split(matrix: np.ndarray[float], labels: np.ndarray[float])->dict[str, float]:
    result_dictionary = {
        "split":None,
        "variance":calculate_variance(labels),
    }
    for column_index in range(matrix.shape[1]):
        column_values = matrix[:, column_index] #select all values from the column of index column_index
        split_value, variance, indexes_1, indexes_2 = calculate_best_column_variance(column_values, labels, result_dictionary['variance'])
        if variance<result_dictionary['variance']:
            result_dictionary['column_index'] = column_index
            result_dictionary['split'] = split_value
            result_dictionary['variance'] = variance
            result_dictionary["X1"] = matrix[indexes_1]
            result_dictionary["X2"] = matrix[indexes_2]
            result_dictionary["y1"] = labels[indexes_1]
            result_dictionary["y2"] = labels[indexes_2]
    return result_dictionary

In [34]:
calculate_matrix_best_split(X_train, y_train)


{'split': 0.005649978676881689,
 'variance': 4288.930696708918,
 'column_index': 2,
 'X1': array([[ 0.06350368, -0.04464164, -0.05039625, ...,  0.02360753,
          0.05803805,  0.04034337],
        [-0.08906294, -0.04464164, -0.01159501, ...,  0.03430886,
          0.02268774, -0.00936191],
        [-0.0382074 , -0.04464164, -0.0105172 , ..., -0.00259226,
         -0.01811369, -0.01764613],
        ...,
        [-0.09632802, -0.04464164, -0.07626374, ..., -0.03949338,
         -0.05947118, -0.08391984],
        [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
         -0.01090325, -0.0010777 ],
        [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
         -0.03845972, -0.03835666]]),
 'X2': array([[-0.00551455, -0.04464164,  0.04229559, ..., -0.03949338,
          0.05227699,  0.02791705],
        [ 0.0090156 , -0.04464164,  0.05522933, ...,  0.02323852,
          0.05568623,  0.10661708],
        [ 0.01628068,  0.05068012,  0.01427248, ...,  0.03430886,
       

In [35]:
import numpy as np

def split_data_best_split(X: np.ndarray, y: np.ndarray):
    tree = {}
    best_split_dct = calculate_matrix_best_split(X, y)
    if best_split_dct["split"] is None:
        tree["value"] = y
    else:
        branch_1 = (best_split_dct["X1"], best_split_dct["y1"])
        branch_2 = (best_split_dct["X2"], best_split_dct["y2"])

        tree['column_index'] = best_split_dct['column_index']
        tree['split'] = best_split_dct['split']
        if len(branch_1[0]) > 1:
            tree['left'] = split_data_best_split(branch_1[0], branch_1[1])
        else:
            tree['left'] = {'value': branch_1[1]}  # Assuming leaf node value
        if len(branch_2[0]) > 1:
            tree['right'] = split_data_best_split(branch_2[0], branch_2[1])
        else:
            tree['right'] = {'value': branch_2[1]}  # Assuming leaf node value

    return tree


In [36]:
res = split_data_best_split(X_train, y_train)

Nice, we were now able to separate the different samples and reduced the variance. If we now want to calculate the label y for an observation we just have to follow the tree and reach the leaf node. When we reach it, we calculate the average of the values of the leaf node and that is the value we want! Let's try

In [37]:
def predict(x:np.ndarray, tree:dict)->float:
    if "value" in tree.keys():
        return np.mean(tree['value'])
    else:
        if x[tree['column_index']]<tree["split"]:
            return predict(x, tree['left'])
        else:
            return predict(x, tree['right'])

In [38]:
predict(X_train[0], res)

166.0

In [39]:
y_train[0]

166.0

As we can see, it's pretty bad, but it works! Why is it so bad? Because it is overfitted with the training data

In [40]:
y_pred = [predict(x, res) for x in X_train]
error_train = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(y_pred, y_train)) / len(X_train)
error_train

0.0

In [41]:
y_pred = [predict(x, res) for x in X_test]
error_test = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(y_pred, y_test)) / len(X_test)
error_test

6103.756756756757

In [42]:
from sklearn.tree import DecisionTreeRegressor
tr = DecisionTreeRegressor(random_state=42)
tr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [43]:
preds = tr.predict(X_train)
error_train = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(preds, y_train)) / len(X_train)
error_train

0.0

In [44]:
preds = tr.predict(X_test)
error_test = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(preds, y_test)) / len(X_test)
error_test

5941.7027027027025